### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import CMNISTDataset
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

cmnist = CMNISTDataset(
    data_root=CFG["datasets"]["CMNIST"]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:
dataset_name = "CMNIST" 
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(f"{dataset_name}_logger")

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

2025-10-10 23:12:18,823 | INFO | Initialized experiment directories for CMNIST
2025-10-10 23:12:18,823 | INFO | Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\CMNIST\logs
2025-10-10 23:12:18,823 | INFO | Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\CMNIST\checkpoints
2025-10-10 23:12:18,823 | INFO | Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\CMNIST\plots


### Setup

In [4]:
domains = CFG["datasets"]["CMNIST"]["domains"]
loaders = {d: {"train": cmnist.get_dataloader(d, train=True), "val": cmnist.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset="CMNIST")
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]

{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key="CMNIST",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

2025-10-10 22:32:10,834 | INFO | === LODO: Leaving out domain 'red' ===



=== LODO: Leaving out domain 'red' ===


Evaluating: 100%|██████████| 50/50 [00:10<00:00,  4.88it/s]
2025-10-10 22:32:39,881 | INFO | [red] Epoch 1/5 | Train - Loss: 0.4737, Cls: 0.4662, GRQO: 0.0074, Acc: 0.8747 | Val - Loss: 0.1957, Cls: 0.1946, GRQO: 0.0012, Acc: 0.9412
2025-10-10 22:32:39,956 | INFO | [red] New best val acc: 0.9412


[red] Epoch 1/5 | Train - Loss: 0.4737, Cls: 0.4662, GRQO: 0.0074, Acc: 0.8747 | Val - Loss: 0.1957, Cls: 0.1946, GRQO: 0.0012, Acc: 0.9412
[red] New best val acc: 0.9412


Evaluating: 100%|██████████| 50/50 [00:10<00:00,  4.97it/s]
2025-10-10 22:33:08,380 | INFO | [red] Epoch 2/5 | Train - Loss: 0.0383, Cls: 0.0357, GRQO: 0.0026, Acc: 0.9899 | Val - Loss: 0.1084, Cls: 0.1080, GRQO: 0.0004, Acc: 0.9668
2025-10-10 22:33:08,449 | INFO | [red] New best val acc: 0.9668


...

[blue] Epoch 3/5 | Train - Loss: 0.0189, Cls: 0.0168, GRQO: 0.0021, Acc: 0.9958 | Val - Loss: 0.2059, Cls: 0.2054, GRQO: 0.0005, Acc: 0.9376
[blue] New best val acc: 0.9376


Evaluating: 100%|██████████| 49/49 [00:11<00:00,  4.30it/s]
2025-10-10 22:39:33,138 | INFO | [blue] Epoch 4/5 | Train - Loss: 0.0094, Cls: 0.0077, GRQO: 0.0017, Acc: 0.9981 | Val - Loss: 0.2201, Cls: 0.2195, GRQO: 0.0005, Acc: 0.9352


[blue] Epoch 4/5 | Train - Loss: 0.0094, Cls: 0.0077, GRQO: 0.0017, Acc: 0.9981 | Val - Loss: 0.2201, Cls: 0.2195, GRQO: 0.0005, Acc: 0.9352


Evaluating: 100%|██████████| 49/49 [00:11<00:00,  4.34it/s]
2025-10-10 22:40:06,069 | INFO | [blue] Epoch 5/5 | Train - Loss: 0.0083, Cls: 0.0069, GRQO: 0.0014, Acc: 0.9980 | Val - Loss: 0.2232, Cls: 0.2228, GRQO: 0.0003, Acc: 0.9358
2025-10-10 22:40:06,069 | INFO | [blue] Best Acc: 0.9376
2025-10-10 22:40:06,069 | INFO | ------------------------------------------------------------
2025-10-10 22:40:06,069 | INFO | LODO finished | Mean Acc: 0.9591 | Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\CMNIST\logs\lodo_summary_20251010_224006.json


[blue] Epoch 5/5 | Train - Loss: 0.0083, Cls: 0.0069, GRQO: 0.0014, Acc: 0.9980 | Val - Loss: 0.2232, Cls: 0.2228, GRQO: 0.0003, Acc: 0.9358
[blue] Best Acc: 0.9376
------------------------------------------------------------
LODO finished | Mean Acc: 0.9591
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\resnet\CMNIST\logs\lodo_summary_20251010_224006.json


### Baseline

In [5]:
model_name ="resnet18"
baseline_results, baseline_mean = run_baseline(
    model_name=model_name,
    CFG=CFG,
    logger=logger,
    dataset_key="CMNIST",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


2025-10-10 23:12:20,991 | INFO | Initializing ResNet baseline: resnet18
2025-10-10 23:12:21,156 | INFO | === Baseline LODO: Leaving out domain 'red' ===


Initializing ResNet baseline: resnet18

=== Baseline LODO: Leaving out domain 'red' ===


2025-10-10 23:12:49,556 | INFO | [red] Epoch 1/5 | Train - Loss: 0.3227, Acc: 0.9206 | Val Acc: 0.9461


[red] Epoch 1/5 | Train - Loss: 0.3227, Acc: 0.9206 | Val Acc: 0.9461


2025-10-10 23:13:15,941 | INFO | [red] Epoch 2/5 | Train - Loss: 0.0364, Acc: 0.9919 | Val Acc: 0.9501


...

[blue] Epoch 3/5 | Train - Loss: 0.0126, Acc: 0.9984 | Val Acc: 0.8096


2025-10-10 23:18:34,890 | INFO | [blue] Epoch 4/5 | Train - Loss: 0.0042, Acc: 0.9997 | Val Acc: 0.6991


[blue] Epoch 4/5 | Train - Loss: 0.0042, Acc: 0.9997 | Val Acc: 0.6991


2025-10-10 23:19:02,137 | INFO | [blue] Epoch 5/5 | Train - Loss: 0.0025, Acc: 0.9999 | Val Acc: 0.7974
2025-10-10 23:19:02,138 | INFO | [blue] Best Val Acc: 0.8096
2025-10-10 23:19:02,138 | INFO | ------------------------------------------------------------
2025-10-10 23:19:02,138 | INFO | Baseline LODO (resnet18) finished | Mean Acc: 0.8605


[blue] Epoch 5/5 | Train - Loss: 0.0025, Acc: 0.9999 | Val Acc: 0.7974
[blue] Best Val Acc: 0.8096
------------------------------------------------------------
Baseline LODO (resnet18) finished | Mean Acc: 0.8605
